# Transforming SymEngine expressions into functions

https://discourse.julialang.org/t/creating-julia-function-from-a-symbolic-expression-using-symengine/21476

In [62]:
using IntervalMatrices
using SymEngine
using MacroTools: postwalk

subidx(i) = join(Char.(0x2080 .+ convert.(UInt16, digits(i)[end:-1:1])))

function symbolic_matrix(n; name="M")
    M = Matrix{SymEngine.Basic}(undef, n, n)
    for i in 1:n, j in 1:n
        M[i, j] = name * subidx(i) * subidx(j)
    end
    return M
end

function subs(ex, imat; name="M")
    n = size(imat, 1) # imat assumed square
    for i in 1:n, j in 1:n
        name_ij = Symbol(name * subidx(i) * subidx(j))
        ex == name_ij && return imat[i, j]
    end
    return ex
end

Msym_eval(Msym, M, k) = [postwalk(ex -> subs(ex, M), convert(Expr, m)) for m in Msym]

# this version gives a more accurate result than just M^k because lots of intermediate
# interval evaluations are saved. however, it is very bad in terms of performance because
# the expression tree is consdered for each term..
function power(M, k)
    n = size(M, 1)
    @assert n == size(M, 2)
    Msym = symbolic_matrix(n)
    X = Msym_eval(Msym^k, M, k)
    Mᵏ = eval.(X)
    return Mᵏ
end

using SymEngine: toString

# this works but it is slow
# it has been reported here: 
# https://discourse.julialang.org/t/creating-julia-function-from-a-symbolic-expression-using-symengine/21476
# using lambify2 didn't help, it gives world age issues (see below)
function power2(M, k)
    n = size(M, 1)
    @assert n == size(M, 2)
    Msym = symbolic_matrix(n)
    Msymᵏ = Msym^k
    vars = Tuple(Msym)
    Msym_callable = map(x -> lambdify(x, vars), Msymᵏ)
    vals = Tuple(M)
    Mᵏ = map(Mij-> Mij(vals...), Msym_callable)
    return Mᵏ
end

#lambdify2(ex, vars=free_symbols(ex)) = eval(Expr(:function, Expr(:call, gensym(), map(Symbol,vars)...),
#                                       convert(Expr, ex)))
#=
MethodError: no method matching ##371(::Interval{Float64}, ::Interval{Float64}, ::Interval{Float64}, ::Interval{Float64})
The applicable method may be too new: running in world age 26092, while current world is 26096.
Closest candidates are:
  ##371(::Any, ::Any, ::Any, ::Any) at none:0 (method too new to be called from this world context.)
=#
lambdify3(ex, vars=free_symbols(ex)) = eval(Meta.parse("ex(x)="*SymEngine.toString(ex...)))

function power3(M, k)
    n = size(M, 1)
    @assert n == size(M, 2)
    Msym = symbolic_matrix(n)
    vars = Tuple(Msym)
    Msymᵏ = Msym^k
    
    vars_str = toString.(vec(Msym))
    vars_str_concat = vars_str[1] * ", " * vars_str[2] * ", " * vars_str[3] * ", " * vars_str[4]

    # create callable entries
    Msymᵏ_callable = []
    for j in 1:n, i in 1:n
        push!(Msymᵏ_callable, eval(Meta.parse("M$i$j("* vars_str_concat * ") = " * SymEngine.toString(Msymᵏ[i, j]))))
    end

    # evaluate them
    vals = Tuple(M)
    Mᵏ = map(Mij-> Mij(vals...), Msymᵏ_callable)
    Mᵏ = reshape(Mᵏ, n, n)
    return Mᵏ
end

power3 (generic function with 1 method)

In [115]:
@time power(M, 10)

  0.069640 seconds (280.31 k allocations: 17.813 MiB, 5.68% gc time)


2×2 Array{Interval{Float64},2}:
 [-1.55487e+06, 2.60516e+06]  [-1.42804e+06, 4.62419e+06]
 [-1.53971e+06, 2.1323e+06]   [-1.75444e+06, 3.78486e+06]

In [118]:
@time power3(M, 10)

 13.895185 seconds (14.31 M allocations: 520.146 MiB, 3.61% gc time)


2×2 Array{Interval{Float64},2}:
 [-1.55487e+06, 2.60516e+06]  [-1.42804e+06, 4.62419e+06]
 [-1.53971e+06, 2.1323e+06]   [-1.75444e+06, 3.78486e+06]

In [71]:
M = rand(IntervalMatrix, 2)
k = 2
n = 2

Msym = symbolic_matrix(n)
vars = Tuple(Msym)
Msymᵏ = Msym^k

2×2 Array{Basic,2}:
   M₂₁*M₁₂ + M₁₁^2  M₁₁*M₁₂ + M₂₂*M₁₂
 M₂₁*M₁₁ + M₂₂*M₂₁    M₂₁*M₁₂ + M₂₂^2

In [84]:
fff = []
for j in 1:n, i in 1:n
    push!(fff, eval(Meta.parse("M$i$j("* vars_str_concat * ") = " * SymEngine.toString(Msymᵏ[i, j]))))
end

In [87]:
fff[2]

M21 (generic function with 1 method)

In [79]:
M22(Interval(1), Interval(2), Interval(3), Interval(4))

[22, 22]

In [81]:
M22

M22 (generic function with 1 method)

In [ ]:
for j in 1:n, i in 1:n
    
end

In [25]:
Msym_callable = map(x -> lambdify3(x, vars), M^2)

2×2 Array{typeof(ex),2}:
 ex  ex
 ex  ex

In [57]:
vars_str = toString.(vec(M))

4-element Array{String,1}:
 "M₁₁"
 "M₂₁"
 "M₁₂"
 "M₂₂"

In [59]:
vars_str_concat = vars_str[1] * ", " * vars_str[2] * ", " * vars_str[3] * ", " * vars_str[4]

"M₁₁, M₂₁, M₁₂, M₂₂"

In [38]:
eval(Meta.parse("tt(x)=" * toString(M[1, 1])))

tt (generic function with 1 method)

In [61]:
eval(Meta.parse("myFunc(" * vars_str_concat * ") = " * SymEngine.toString(M[1, 1])))

myFunc (generic function with 1 method)

In [64]:
myFunc(vec(M1)...)

[-0.754373, 0.114535]

In [30]:
Msym_callable[1, 1]((1.0, 1.0, 2.0, 1.0))

UndefVarError: UndefVarError: M₂₁ not defined

In [14]:
M = symbolic_matrix(2)

2×2 Array{Basic,2}:
 M₁₁  M₁₂
 M₂₁  M₂₂

In [15]:
M1 = rand(IntervalMatrix, 2)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.754373, 0.114535]  [-0.34586, 0.340555] 
 [-1.35401, 0.556095]   [-0.262811, 0.720356]

In [16]:
power2(M1, 2)

MethodError: MethodError: no method matching ex(::Interval{Float64}, ::Interval{Float64}, ::Interval{Float64}, ::Interval{Float64})
Closest candidates are:
  ex(::Any) at none:1 (method too new to be called from this world context.)

In [18]:
M * M

2×2 Array{Basic,2}:
   M₂₁*M₁₂ + M₁₁^2  M₁₁*M₁₂ + M₂₂*M₁₂
 M₂₁*M₁₁ + M₂₂*M₂₁    M₂₁*M₁₂ + M₂₂^2

In [66]:
g(k) = map(x -> lambdify(x, vec(M)), M^k)

g (generic function with 1 method)

In [68]:
g(2)

2×2 Array{getfield(SymEngine, Symbol("##28#29")),2}:
 ##28#29{###484}(##484)  ##28#29{###486}(##486)
 ##28#29{###485}(##485)  ##28#29{###487}(##487)

In [69]:
g(2)[1, 1](vec(M1)...)

[0.223956, 2.02615]

In [75]:
(M * M)[1, 1]

M₂₁*M₁₂ + M₁₁^2

In [77]:
M[2, 1] * M[1, 2] + M[1, 1]^2

M₂₁*M₁₂ + M₁₁^2

In [78]:
M1[2, 1] * M1[1, 2] + M1[1, 1]^2

[0.223956, 2.02615]

In [82]:
M1[2, 1] * M1[1, 2] + M1[1, 1] * M1[1, 1] # la diferencia es que usar ^ es mejor

[-0.611018, 2.02615]

In [80]:
M1 * M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.611018, 2.02615]  [-0.659428, 0.937166]
 [-1.46068, 2.09986]    [0.227325, 1.32085] 

In [89]:
map(q -> q(vec(M1)...), g(10))

2×2 Array{Interval{Float64},2}:
  [-76.7823, 183.854]  [-67.387, 95.769]  
 [-149.267, 214.585]   [-65.8416, 111.778]

In [86]:
M1^10

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-109.056, 183.854]  [-67.387, 95.769]  
 [-149.267, 214.585]  [-72.3784, 111.778]

In [62]:
M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.866816, 0.963267]  [-0.700112, -0.257882] 
 [-1.56871, -0.868445]  [-0.471779, -0.0580359]

In [83]:
M1^2

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.611018, 2.02615]  [-0.659428, 0.937166]
 [-1.46068, 2.09986]    [0.227325, 1.32085] 

In [91]:
power(M1, 10)

2×2 Array{Interval{Float64},2}:
  [-76.7823, 183.854]  [-67.387, 95.769]  
 [-149.267, 214.585]   [-65.8416, 111.778]

In [96]:
M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.234543, 0.676057]   [0.150208, 0.681045]
  [0.0248801, 1.0225]   [-0.847483, 1.86767] 

In [96]:
@time power(M1, 2)

  0.167249 seconds (281.08 k allocations: 14.620 MiB, 3.49% gc time)


2×2 Array{Interval{Float64},2}:
 [-0.461112, 1.03738]  [-0.506048, 0.506228]
 [-1.39487, 1.42201]   [-0.461112, 0.987208]

In [97]:
@time power2(M1, 2)

  0.232383 seconds (446.45 k allocations: 21.823 MiB, 2.14% gc time)


2×2 Array{Interval{Float64},2}:
 [-0.461112, 1.03738]  [-0.506048, 0.506228]
 [-1.39487, 1.42201]   [-0.461112, 0.987208]

In [98]:
@time power3(M, 2)

  0.091020 seconds (90.56 k allocations: 3.727 MiB)


4-element Array{Interval{Float64},1}:
 [-0.215565, 2.43122]
 [-0.96053, 1.33022] 
  [0.290226, 2.88476]
  [0.710786, 3.16717]

In [102]:
power3(M, 10)

4-element Array{Interval{Float64},1}:
 [-577.491, 1030.79]
 [-609.014, 843.411]
 [-448.284, 1829.06]
 [-641.404, 1497.41]

In [101]:
reshape(power3(M, 10), 2, 2)

2×2 Array{Interval{Float64},2}:
 [-577.491, 1030.79]  [-448.284, 1829.06]
 [-609.014, 843.411]  [-641.404, 1497.41]

# Using TaylorSeries

In [30]:
using TaylorSeries, IntervalMatrices

In [126]:
M1 = rand(IntervalMatrix, 2)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.133024, 1.42048]  [-1.48836, 0.56276] 
 [-1.57208, 0.247856]   [0.130534, 1.84444]

In [127]:
#vars = set_variables(Interval{Float64}, "M", numvars=4, order=30);
vars = set_variables(Float64, "M", numvars=4, order=30);

In [128]:
M = [vars[1] vars[2]; vars[3] vars[4]]

2×2 Array{TaylorN{Float64},2}:
  1.0 M₁ + 𝒪(‖x‖³¹)   1.0 M₂ + 𝒪(‖x‖³¹)
  1.0 M₃ + 𝒪(‖x‖³¹)   1.0 M₄ + 𝒪(‖x‖³¹)

In [136]:
evaluate((M^10)[1, 1], vec(M1))

[-2557.93, 45147.4]

In [134]:
(M1*M1)[1, 1]

[-0.867007, 4.35756]

In [137]:
power(M1, 10)

2×2 Array{Interval{Float64},2}:
 [-13891, 45147.4]           [-50436.3, 19070.4]
        [-53273.5, 10768.6]  [-15222.4, 59514.4]

In [72]:
M10 = M^10;

In [74]:
M1 = rand(IntervalMatrix, 2);

In [75]:
evaluate(M10[1, 1], [1.0, 2.0, 3.0, 4.0])

[4.7838e+06, 4.78381e+06]

In [41]:
evaluate(M10[1, 1], vec(M1))

[-13.4182, 19.8009]

In [43]:
(M1^10)[1, 1]

[-14.5554, 14.5198]

In [170]:
function power4(M, k)
    n = size(M, 1)
    @assert n == size(M, 2)

    # the order should be sufficiently high (>= k ?)
    
    vars = set_variables("M", numvars=n^2, order=k+1) # coeffs are float
    #vars = set_variables(Interval{Float64}, "M", numvars=n^2, order=k+1) # coeffs are intervals
    
    Msym = reshape(vars, n, n)
    Msymᵏ = Msym^k

    vals = vec(M)
    Mᵏ = map(Mij -> evaluate(Mij, vals), Msymᵏ)
    
    return Mᵏ
end

power4 (generic function with 1 method)

In [100]:
M1

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.706496, -0.292591]  [0.0637538, 0.979007]
  [0.190738, 0.89017]    [0.377551, 2.38851]  

In [101]:
power4(M1, 2)

2×2 Array{Interval{Float64},2}:
  [0.09777, 1.37062]   [-0.667594, 2.31972]
 [-0.556888, 2.07037]   [0.154705, 6.57645]

AssertionError: AssertionError: order ≤ get_order() && lencoef ≤ num_coeffs

In [148]:
M1 = rand(IntervalMatrix, 4)

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.87559, -0.544873]    [0.00564368, 0.723806]  …  [-1.44069, 0.692388] 
 [-0.209292, 0.983093]   [-0.212949, 0.499019]        [0.541998, 2.04747] 
 [-0.271004, -0.169206]  [-0.48511, 0.241644]        [-1.77807, 0.155637] 
  [1.08192, 2.25097]     [-0.941258, -0.0850188]     [-1.98761, -0.978445]

In [161]:
@time power(M1, 5)

  0.167815 seconds (1.76 M allocations: 117.401 MiB, 11.05% gc time)


4×4 Array{Interval{Float64},2}:
 [-492.021, 505.052]  [-287.967, 281.671]  …  [-571.821, 816.928]
 [-365.218, 464.079]  [-264.769, 220.174]     [-618.709, 488.785]
 [-429.105, 374.359]  [-217.951, 248.477]     [-465.468, 622.849]
 [-796.099, 544.992]  [-321.434, 461.372]     [-957.546, 938.641]

In [162]:
@time power4(M1, 5)

  0.289164 seconds (943.99 k allocations: 648.964 MiB, 14.13% gc time)


4×4 Array{Interval{Float64},2}:
 [-667.419, 639.79]   [-375.194, 409.248]  …   [-877.456, 1106.53]
 [-446.234, 526.3]    [-317.758, 276.425]      [-862.574, 679.32] 
 [-495.931, 441.971]  [-272.3, 303.099]        [-640.677, 849.931]
 [-900.944, 648.923]  [-427.766, 522.825]     [-1248.05, 1206.07] 

In [167]:
@time power4(M1, 8)

InterruptException: InterruptException:

In [173]:
using BenchmarkTools

In [176]:
M1 = rand(IntervalMatrix, 2)

2×2 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.348735, -0.00446299]  [-2.00009, 0.403893]
 [-2.67661, 0.237629]      [-0.358738, 1.96379]

In [181]:
@btime power($M1, 4)

  1.736 ms (9875 allocations: 640.95 KiB)


2×2 Array{Interval{Float64},2}:
  [-7.95888, 51.6993]  [-59.7474, 21.8903] 
 [-78.4055, 27.8079]    [-7.95888, 106.546]

In [185]:
@btime power4($M1, 4)

  176.546 μs (4471 allocations: 246.99 KiB)


2×2 Array{Interval{Float64},2}:
 [-11.8961, 52.7535]  [-58.9023, 21.0803]
 [-79.5365, 26.9885]  [-19.9713, 107.6]  

In [183]:
@btime power($M1, 10)

  10.921 ms (78404 allocations: 4.98 MiB)


2×2 Array{Interval{Float64},2}:
 [-27892, 90762]     [-117012, 42870.7]        
 [-153552, 54459.8]          [-35831.5, 198175]

In [184]:
@btime power4($M1, 10)

  1.224 ms (20083 allocations: 1.76 MiB)


2×2 Array{Interval{Float64},2}:
         [-34890.7, 93058.6]  [-115634, 42072]
 [-155225, 57504.5]           [-53331, 200012]

In [186]:
power4(M1, 15)

2×2 Array{Interval{Float64},2}:
 [-2.61366e+07, 5.07907e+07]  [-6.4509e+07, 2.88482e+07] 
 [-8.63974e+07, 3.87699e+07]  [-4.34392e+07, 1.08748e+08]

## Using DynamicPolynomials

In [187]:
using DynamicPolynomials

# this version is as fast as M^k where M is an inteval, but it is not accurate. the reason is that 
# DynamicPolynomials expands the symbolic polynomial products 
function power_poly(M::IntervalMatrix, k::Int)
    n = size(M, 1)
    @polyvar Msym[1:n, 1:n]
    vars = vec(Msym) # symbolic variables, column-major
    Msym_k = Msym^k # symbolic matrix multiplication << can we "hold" this product?
    Mk = map(x -> subs(x, vars => vec(M)), Msym_k) # substitution of the intervals
    return Mk
end

┌ Info: Recompiling stale cache file /home/mforets/.julia/compiled/v1.2/DynamicPolynomials/tepBk.ji for DynamicPolynomials [7c1d4256-1411-5781-91ec-d7bc3513ac07]
└ @ Base loading.jl:1240


power_poly (generic function with 1 method)

In [194]:
M = rand(IntervalMatrix, 2)
k = 2

2

In [198]:
n = size(M, 1)
@polyvar Msym[1:n, 1:n]
vars = vec(Msym) # symbolic variables, column-major
Msym_k = Msym^k # symbolic matrix multiplication << can we "hold" this product?
Mk = map(x -> subs(x, vars => vec(M)), Msym_k) # substitution of the intervals

MethodError: MethodError: no method matching size(::Pair{Array{PolyVar{true},1},Base.ReshapedArray{Interval{Float64},1,IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}},Tuple{}}}, ::Int64)
Closest candidates are:
  size(!Matched::BitArray{1}, ::Integer) at bitarray.jl:81
  size(!Matched::Tuple, ::Integer) at tuple.jl:22
  size(!Matched::Number, ::Integer) at number.jl:63
  ...

In [209]:
vars = vec(Msym)

4-element Array{PolyVar{true},1}:
 Msym₁₋₁
 Msym₂₋₁
 Msym₁₋₂
 Msym₂₋₂

In [210]:
subs(Msym_k[1, 1], vars => Vector(vec(M1)))

MethodError: MethodError: no method matching size(::Pair{Array{PolyVar{true},1},Array{Interval{Float64},1}}, ::Int64)
Closest candidates are:
  size(!Matched::BitArray{1}, ::Integer) at bitarray.jl:81
  size(!Matched::Tuple, ::Integer) at tuple.jl:22
  size(!Matched::Number, ::Integer) at number.jl:63
  ...

In [189]:
power_poly(M1, 10)

MethodError: MethodError: no method matching size(::Pair{Array{PolyVar{true},1},Base.ReshapedArray{Interval{Float64},1,IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}},Tuple{}}}, ::Int64)
Closest candidates are:
  size(!Matched::BitArray{1}, ::Integer) at bitarray.jl:81
  size(!Matched::Tuple, ::Integer) at tuple.jl:22
  size(!Matched::Number, ::Integer) at number.jl:63
  ...

In [ ]:
@btime power_poly

# Changing powers to exp

1) Input: interval matrix M of order n, integer k >= 1, and truncation order p

2) Output: the sequence {Q, Q2, ..., Qk} such that Qi is an overapproximation of (exp(M))^i for each i = 1, ..., k

Algorithm 1

```
Q <- exp_oa(M, p)
Q2 <- Q * Q
Q3 <- Q * Q2
...
Qk <- Q * Q_{k-1}
```

Algorithm 2

```
Q <- exp_oa(M, p)
Q2 <- exp_oa(2*M, p)
...
Qk <- exp_oa(k*M, p)
```

---

We implement Algorithm 2 next.

In [31]:
using BenchmarkTools
using Revise, IntervalMatrices

In [19]:
# return exp(M), exp(2M), ..., exp(kM) with overapproximation of order p
function pexp2(M, k, p)
    out = Vector{IntervalMatrix{Float64}}(undef, k)
    for i in 1:k
        out[i] = exp_overapproximation(i*M, 1.0, p)
    end
    return out
end

pexp2 (generic function with 1 method)

Let's compare with a naive form of Algorithm 1, where the powers of intervals are taken as if they were dense matrices.

In [32]:
function pexp1(M, k, p)
    out = Vector{IntervalMatrix{Float64}}(undef, k)
    phi1 = exp_overapproximation(M, 1.0, p)
    out[1] = copy(phi1)
    for i in 2:k
        out[i] = out[i-1] * phi1
    end
    return out
end

pexp1 (generic function with 1 method)

### Test

In [145]:
M = rand(IntervalMatrix, 40) * 0.1

40×40 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-0.173159, 0.113827]      …  [-0.0239861, 0.115224]  
 [-0.0019215, 0.148086]        [-0.0493541, 0.0639193] 
  [0.0425522, 0.0689519]       [-0.111356, -0.0328481] 
 [-0.200982, -0.0328199]       [-0.139579, 0.12546]    
 [-0.160794, -0.000903328]      [0.0311427, 0.105935]  
 [-0.098791, 0.0597009]     …   [0.0420112, 0.143352]  
 [-0.0196645, 0.275741]        [-0.0564964, 0.0498044] 
 [-0.108853, 0.0575663]        [-0.123532, 0.0110838]  
 [-0.092545, 0.000252951]       [0.0960279, 0.102825]  
 [-0.0668071, 0.0216959]       [-0.216341, -0.0512607] 
 [-0.162773, -0.0354169]    …   [0.00425128, 0.196969] 
 [-0.0878364, -0.022894]       [-0.0608379, 0.208074]  
 [-0.0700484, 0.133077]        [-0.11197, 0.139085]    
   ⋮                        ⋱                          
  [0.0727462, 0.0774259]       [-0.0921877, 0.0871373] 
 [-0.10507, 0.221231]          [-0.130604, -0.111618]  
 [-0.054951, 0.0369961]     

In [148]:
@time out1 = pexp1(M, 100, 30);

  0.370830 seconds (1.46 k allocations: 6.462 MiB)


In [146]:
@time out2 = pexp2(M, 100, 30);

 13.798987 seconds (57.37 k allocations: 408.562 MiB, 0.24% gc time)


In [155]:
out1[5][1, 1]

[-1.08212e+24, 1.08212e+24]

In [152]:
out2[end][1, 1]

[-3.75895e+196, 3.75895e+196]

In [128]:
out1[1]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.846621, 1.07392]     …  [-0.0952449, 0.0607917]
 [-0.0455161, 0.0490107]     [-0.167436, 0.0829671] 
 [-0.159921, 0.0308832]      [-0.0471625, 0.101086] 
 [-0.0488102, 0.0477423]      [0.88971, 1.02042]    

In [129]:
out2[1]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.846621, 1.07392]     …  [-0.0952449, 0.0607917]
 [-0.0455161, 0.0490107]     [-0.167436, 0.0829671] 
 [-0.159921, 0.0308832]      [-0.0471625, 0.101086] 
 [-0.0488102, 0.0477423]      [0.88971, 1.02042]    

In [130]:
out1[2]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.685072, 1.19588]    [-0.559099, -0.0417085]  …  [-0.236701, 0.177821]
 [-0.111827, 0.139412]    [0.660913, 1.24785]        [-0.375751, 0.187764]
 [-0.367969, 0.0793805]  [-0.191226, 0.298916]       [-0.13129, 0.249365] 
 [-0.145427, 0.111953]   [-0.0225697, 0.230179]       [0.760428, 1.07809] 

In [131]:
out2[2]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.67165, 1.20071]     [-0.572507, -0.0154382]  …  [-0.253585, 0.186078]
 [-0.122137, 0.143756]    [0.648252, 1.25635]        [-0.384809, 0.196241]
 [-0.370156, 0.0873174]  [-0.203323, 0.299602]       [-0.135754, 0.252503]
 [-0.147659, 0.122127]   [-0.036567, 0.237018]        [0.75199, 1.08547]  

In [132]:
out1[3]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.498957, 1.3847]    [-0.974183, -0.012191]  …  [-0.44191, 0.374526] 
 [-0.207093, 0.292722]   [0.443099, 1.45231]       [-0.645778, 0.324508]
 [-0.648069, 0.155665]  [-0.35455, 0.599156]       [-0.273371, 0.465088]
 [-0.305982, 0.199724]  [-0.0914398, 0.446493]      [0.601786, 1.18786] 

In [133]:
out2[3]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [0.437279, 1.41383]   [-1.03797, 0.107389]   …  [-0.519792, 0.415745]
 [-0.25732, 0.315041]    [0.384145, 1.49463]      [-0.687255, 0.368618]
 [-0.659323, 0.193761]  [-0.408671, 0.605324]     [-0.295731, 0.4841]  
 [-0.320246, 0.247937]  [-0.154346, 0.478671]      [0.559887, 1.22333] 

In [134]:
out1[10]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
  [-7.19539, 9.59238]  [-14.0893, 7.80281]   …  [-8.13595, 10.0302]
  [-4.21488, 8.6107]    [-9.9608, 9.14884]      [-9.58146, 5.55632]
 [-10.7401, 4.82381]    [-8.15425, 14.6714]     [-8.27474, 9.38061]
  [-7.59119, 3.90841]   [-5.38982, 10.6257]     [-5.08521, 7.34588]

In [135]:
out2[10]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-40.1939, 42.3004]  [-57.9744, 54.1517]  …  [-46.8402, 46.2157]
 [-35.9377, 37.2392]  [-48.5049, 49.6182]     [-42.3154, 39.9235]
 [-37.3024, 34.2569]  [-47.0701, 49.4001]     [-39.1526, 40.183] 
 [-34.9855, 33.7607]  [-44.8888, 47.057]      [-37.0285, 39.0208]

In [136]:
out1[end]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-9.5126e+07, 9.51711e+07]   …  [-1.03181e+08, 1.03189e+08]
 [-7.91068e+07, 7.91048e+07]     [-8.58217e+07, 8.57523e+07]
 [-9.31189e+07, 9.30657e+07]     [-1.00923e+08, 1.00987e+08]
 [-6.66761e+07, 6.66397e+07]     [-7.22629e+07, 7.23126e+07]

In [137]:
out2[end]

4×4 IntervalMatrix{Float64,Interval{Float64},Array{Interval{Float64},2}}:
 [-1.70538e+22, 1.70538e+22]  …  [-1.89204e+22, 1.89204e+22]
 [-1.64003e+22, 1.64003e+22]     [-1.81953e+22, 1.81953e+22]
 [-1.62182e+22, 1.62182e+22]     [-1.79933e+22, 1.79933e+22]
 [-1.60717e+22, 1.60717e+22]     [-1.78307e+22, 1.78307e+22]